# 01: MERGE Preprocessing

This notebook preprocesses the MERGE balanced dataset for downstream embedding and clustering tasks. 

The goal is to load metadata and arousal-valence annotations, filter for complete records, and produce a clean subset with validated audio paths.

## Setup

In [6]:
import sys
from pathlib import Path

import pandas as pd

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

from configs.dataset import load_config
from dataset.filter import METADATA_COLUMNS, AV_COLUMNS

In [7]:
config = load_config(project_root / "configs" / "config.yaml")
merge = config.datasets["merge"]

print(f"Dataset root: {merge.root}")
print(f"Audio dir: {merge.audio.dir}")

Dataset root: /Users/kat/Desktop/code/projects/data/merge-balanced
Audio dir: /Users/kat/Desktop/code/projects/data/merge-balanced/audio


## Data Preparation

### Load raw CSVs

The MERGE dataset provides two CSV files:
- `merge_audio_balanced_metadata.csv` with 14 columns of track metadata and tags
- `merge_audio_balanced_av_values.csv` with arousal-valence annotations

In [8]:
metadata_path = merge.metadata["metadata_file"]
av_path = merge.metadata["av_values_file"]

meta_df = pd.read_csv(metadata_path)
av_df = pd.read_csv(av_path)

print(f"Metadata: {len(meta_df)} rows, {len(meta_df.columns)} columns")
print(f"AV values: {len(av_df)} rows, {len(av_df.columns)} columns")

Metadata: 3232 rows, 26 columns
AV values: 3232 rows, 3 columns


### Standardize column names

We use the column mappings from `dataset.filter` to convert raw headers to standardized names.

In [9]:
meta_df = meta_df.rename(columns=METADATA_COLUMNS)
meta_df = meta_df[list(METADATA_COLUMNS.values())]

av_df = av_df.rename(columns=AV_COLUMNS)
av_df = av_df[list(AV_COLUMNS.values())]

print("Metadata columns:", list(meta_df.columns))
print("AV columns:", list(av_df.columns))

Metadata columns: ['song_id', 'quadrant', 'artist', 'title', 'duration', 'mood', 'mood_all', 'mood_all_weights', 'genre', 'genre_weights', 'theme', 'theme_weights', 'style', 'style_weights']
AV columns: ['song_id', 'arousal', 'valence']


### Merge metadata with AV values

Inner join on `song_id` to combine metadata with arousal-valence annotations.

In [10]:
df = meta_df.merge(av_df, on="song_id", how="inner")
print(f"Merged: {len(df)} rows")

Merged: 3232 rows


### Filter for complete data

Require all tag fields to be populated (non-empty strings) and valid arousal/valence values.

In [11]:
tag_cols = ["mood_all", "genre", "theme", "style"]

for col in tag_cols:
    df = df[df[col].notna() & (df[col].str.strip() != "")]

df = df[df["arousal"].notna() & df["valence"].notna()]

print(f"After filtering: {len(df)} rows")

After filtering: 2577 rows


### Build and validate audio paths

Construct audio file paths and filter to tracks with existing files.

In [12]:
audio_dir = merge.audio.dir

def build_audio_path(row: pd.Series) -> Path:
    return audio_dir / row["quadrant"] / f"{row['song_id']}.mp3"

df["audio_path"] = df.apply(build_audio_path, axis=1)
df["audio_exists"] = df["audio_path"].apply(lambda p: p.exists())

n_missing = (~df["audio_exists"]).sum()
print(f"Missing audio files: {n_missing}")

df = df[df["audio_exists"]].drop(columns=["audio_exists"])
print(f"Final: {len(df)} tracks with valid audio")

Missing audio files: 0
Final: 2577 tracks with valid audio


In [13]:
df["audio_path"] = df["audio_path"].astype(str)
df.head()

,song_id,quadrant,artist,title,duration,mood,mood_all,mood_all_weights,genre,genre_weights,theme,theme_weights,style,style_weights,arousal,valence,audio_path
1,A002,Q4,Rod Stewart,Country Comfort,282.0,"Agreeable,Positive,Relaxed,Romantic,Serious,St...","Agreeable,Positive,Relaxed,Romantic,Serious,St...","5,5,5,5,5,5,5,5",Pop/Rock,5,"Biographical,Country Life,Family,Lifecycle,Ope...","5,5,5,5,5,5,5,5","Adult Contemporary,Contemporary Pop/Rock","5,5",0.3750,0.7125,/Users/kat/Desktop/code/projects/data/merge-ba...
8,A014,Q1,Jamiroquai,Feels Just Like It Should,274.0,"Bright,Carefree,Celebratory,Effervescent,Energ...","Bright,Carefree,Celebratory,Effervescent,Energ...","5,6,7,7,7,7,7,7,7,7,7,7,8,8,8,8","Electronic,Pop/Rock,R&B","6,7,9","Club,Day Driving,Partying,Pool Party,TGIF","6,7,7,7,7","Acid Jazz,Adult Alternative Pop/Rock,Alternati...","5,6,7,9,9,9",0.9000,0.7125,/Users/kat/Desktop/code/projects/data/merge-ba...
59,A090-94,Q2,2Pac,Fuck the World,253.0,"Angry,Angst-Ridden,Anguished/Distraught,Comple...","Angry,Angst-Ridden,Anguished/Distraught,Broodi...","5,5,5,5,5,5,5,5,5,5,5,5,7,7,7,7,8,8,8,9,9,9,9",Rap,8,"Affirmation,Cool & Cocky,Empowering,Introspect...","5,5,5,6,7,7,7","G-Funk,Gangsta Rap,West Coast Rap","8,8,8",0.7875,0.1875,/Users/kat/Desktop/code/projects/data/merge-ba...
82,A120-168,Q4,Enya,Paint the Sky With Stars,255.0,"Atmospheric,Calm/Peaceful,Circular,Complex,Det...","Atmospheric,Calm/Peaceful,Circular,Complex,Det...","8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,...","International,New Age,Pop/Rock","8,8,8","Introspection,Meditation,Reflection,Relaxation...","8,8,9,9,9,9","Adult Alternative,Adult Alternative Pop/Rock,A...","8,8,8,8,8",0.0875,0.7500,/Users/kat/Desktop/code/projects/data/merge-ba...
99,A148-102,Q1,Billy Joel,Uptown Girl,197.0,"Amiable/Good-Natured,Brash,Bravado,Bright,Chee...","Amiable/Good-Natured,Brash,Bravado,Bright,Chee...","5,7,9,9,9,9,9,9,9,9,9,9",Pop/Rock,9,"In Love,Joy,New Love","5,9,9","Album Rock,Contemporary Pop/Rock,Soft Rock","9,9,9",0.8375,0.8500,/Users/kat/Desktop/code/projects/data/merge-ba...


## Save Data

In [14]:
output_dir = project_root / "notebooks" / "data"
output_dir.mkdir(exist_ok=True)

output_path = output_dir / "merge_preprocessed.csv"
df.to_csv(output_path, index=False)

print(f"Saved: {output_path}")
print(f"Shape: {df.shape}")

Saved: /Users/kat/Desktop/code/projects/soundspace/notebooks/data/merge_preprocessed.csv
Shape: (2577, 17)
